In [1]:
import scvelo as scv
import loompy
import scanpy
import igraph
import phate
import glob, os
import pandas as pd
import os
import re
import subprocess
from collections import Counter
import numpy as np

ifile = "Hongze_healthy_2020_10_downsampled_1000_2D_overclustered.h5ad"

In [24]:
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

scvelo==0.2.4
loompy==3.0.6
scanpy==1.8.2
igraph==0.9.9
phate==1.0.7
pandas==1.5.0.dev0+268.gbe8d1ec88
re==2.2.1
numpy==1.19.5


In [19]:
ofile = "Hongze_healthy_2020_10_downsampled_1000_2D_overclustered_doubletsRemoved.h5ad"

In [2]:
adata = scv.read(ifile)
adata

AnnData object with n_obs × n_vars = 25067 × 21269
    obs: 'TotalUMIs', 'sname', 'n_counts', 'louvain', 'velocity_self_transition', 'velocity_clusters15', 'louvain15_merged.92', 'louvain15_merged.92_renamed', 'louvain15_merged.95', 'louvain15_merged.95_renamed', 'louvain15_merged.98', 'louvain15_merged.98_renamed', 'louvain11', 'louvain11_merged.95', 'louvain11_merged.95_renamed'
    var: 'Accession', 'AccessionVersion', 'Aliases', 'CcdsID', 'Chromosome', 'ChromosomeEnd', 'ChromosomeStart', 'CosmicID', 'DnaBindingDomain', 'FullName', 'GeneType', 'HgncID', 'IsTF', 'Location', 'LocationSortable', 'LocusGroup', 'LocusType', 'MgdID', 'MirBaseID', 'OmimID', 'PubmedID', 'RefseqID', 'RgdID', 'UcscID', 'UniprotID', 'VegaID', 'n_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'fit_r2', 'fit_alpha', 'fit_beta', 'fit_gamma', 'fit_t_', 'fit_scaling', 'fit_std_u', 'fit_std_s', 'fit_likelihood', 'fit_u0', 'fit_s0', 'fit_pval_steady', 'fit_steady_u', 'fit_steady_s', 'fit_vari

In [3]:
scanpy.external.pp.scrublet(adata)

Automatically set threshold at doublet score = 0.31
Detected doublet rate = 2.5%
Estimated detectable doublet fraction = 53.5%
Overall doublet rate:
	Expected   = 5.0%
	Estimated  = 4.7%


In [4]:
adata

AnnData object with n_obs × n_vars = 25067 × 21269
    obs: 'TotalUMIs', 'sname', 'n_counts', 'louvain', 'velocity_self_transition', 'velocity_clusters15', 'louvain15_merged.92', 'louvain15_merged.92_renamed', 'louvain15_merged.95', 'louvain15_merged.95_renamed', 'louvain15_merged.98', 'louvain15_merged.98_renamed', 'louvain11', 'louvain11_merged.95', 'louvain11_merged.95_renamed', 'doublet_score', 'predicted_doublet'
    var: 'Accession', 'AccessionVersion', 'Aliases', 'CcdsID', 'Chromosome', 'ChromosomeEnd', 'ChromosomeStart', 'CosmicID', 'DnaBindingDomain', 'FullName', 'GeneType', 'HgncID', 'IsTF', 'Location', 'LocationSortable', 'LocusGroup', 'LocusType', 'MgdID', 'MirBaseID', 'OmimID', 'PubmedID', 'RefseqID', 'RgdID', 'UcscID', 'UniprotID', 'VegaID', 'n_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'fit_r2', 'fit_alpha', 'fit_beta', 'fit_gamma', 'fit_t_', 'fit_scaling', 'fit_std_u', 'fit_std_s', 'fit_likelihood', 'fit_u0', 'fit_s0', 'fit_pval_steady', 'fi

In [8]:
counts = adata.obs[['louvain11_merged.95_renamed', 'predicted_doublet' ]].value_counts()
counts

louvain11_merged.95_renamed  predicted_doublet
36                           False                4609
5                            False                3575
16                           False                2306
7                            False                1923
28                           False                1383
                                                  ... 
0                            True                    1
13                           True                    1
11                           True                    1
4                            True                    1
41                           True                    1
Length: 74, dtype: int64

In [13]:
counts[1]

3575

In [14]:
OK = { key : value for key, value  in counts.items() if key[1] == False }
OK

{('36', False): 4609,
 ('5', False): 3575,
 ('16', False): 2306,
 ('7', False): 1923,
 ('28', False): 1383,
 ('27', False): 1293,
 ('0', False): 961,
 ('21', False): 942,
 ('40', False): 879,
 ('17', False): 632,
 ('1', False): 601,
 ('31', False): 558,
 ('19', False): 483,
 ('18', False): 429,
 ('11', False): 255,
 ('39', False): 252,
 ('2', False): 250,
 ('32', False): 244,
 ('3', False): 223,
 ('22', False): 221,
 ('41', False): 212,
 ('33', False): 202,
 ('38', False): 186,
 ('13', False): 184,
 ('4', False): 179,
 ('8', False): 178,
 ('25', False): 170,
 ('12', False): 142,
 ('26', False): 121,
 ('15', False): 110,
 ('20', False): 110,
 ('30', False): 91,
 ('9', False): 75,
 ('14', False): 74,
 ('34', False): 66,
 ('29', False): 62,
 ('10', False): 60,
 ('24', False): 58,
 ('6', False): 50,
 ('23', False): 50,
 ('35', False): 26,
 ('37', False): 9}

In [15]:
dublets = { key : value for key, value  in counts.items() if key[1] == False }
dublets

{('36', False): 4609,
 ('5', False): 3575,
 ('16', False): 2306,
 ('7', False): 1923,
 ('28', False): 1383,
 ('27', False): 1293,
 ('0', False): 961,
 ('21', False): 942,
 ('40', False): 879,
 ('17', False): 632,
 ('1', False): 601,
 ('31', False): 558,
 ('19', False): 483,
 ('18', False): 429,
 ('11', False): 255,
 ('39', False): 252,
 ('2', False): 250,
 ('32', False): 244,
 ('3', False): 223,
 ('22', False): 221,
 ('41', False): 212,
 ('33', False): 202,
 ('38', False): 186,
 ('13', False): 184,
 ('4', False): 179,
 ('8', False): 178,
 ('25', False): 170,
 ('12', False): 142,
 ('26', False): 121,
 ('15', False): 110,
 ('20', False): 110,
 ('30', False): 91,
 ('9', False): 75,
 ('14', False): 74,
 ('34', False): 66,
 ('29', False): 62,
 ('10', False): 60,
 ('24', False): 58,
 ('6', False): 50,
 ('23', False): 50,
 ('35', False): 26,
 ('37', False): 9}

In [16]:
Counter( adata.obs['predicted_doublet'])

Counter({False: 24434, True: 633})

In [17]:
adata[ adata.obs['predicted_doublet'] == False ]

View of AnnData object with n_obs × n_vars = 24434 × 21269
    obs: 'TotalUMIs', 'sname', 'n_counts', 'louvain', 'velocity_self_transition', 'velocity_clusters15', 'louvain15_merged.92', 'louvain15_merged.92_renamed', 'louvain15_merged.95', 'louvain15_merged.95_renamed', 'louvain15_merged.98', 'louvain15_merged.98_renamed', 'louvain11', 'louvain11_merged.95', 'louvain11_merged.95_renamed', 'doublet_score', 'predicted_doublet'
    var: 'Accession', 'AccessionVersion', 'Aliases', 'CcdsID', 'Chromosome', 'ChromosomeEnd', 'ChromosomeStart', 'CosmicID', 'DnaBindingDomain', 'FullName', 'GeneType', 'HgncID', 'IsTF', 'Location', 'LocationSortable', 'LocusGroup', 'LocusType', 'MgdID', 'MirBaseID', 'OmimID', 'PubmedID', 'RefseqID', 'RgdID', 'UcscID', 'UniprotID', 'VegaID', 'n_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'fit_r2', 'fit_alpha', 'fit_beta', 'fit_gamma', 'fit_t_', 'fit_scaling', 'fit_std_u', 'fit_std_s', 'fit_likelihood', 'fit_u0', 'fit_s0', 'fit_pval_stea

In [18]:
adata = adata[ adata.obs['predicted_doublet'] == False ]

In [20]:
if os.path.exists(ofile):
    os.remove(ofile)
adata.write(ofile)
print(ofile)

Hongze_healthy_2020_10_downsampled_1000_2D_overclustered_doubletsRemoved.h5ad


In [22]:
del adata.uns['scrublet']

In [23]:
if os.path.exists(ofile):
    os.remove(ofile)
adata.write(ofile)
print(ofile)

Hongze_healthy_2020_10_downsampled_1000_2D_overclustered_doubletsRemoved.h5ad
